In [27]:
import os
import pandas as pd
import imageio

In [32]:
def rotation_judge(video_path, csv_path, output_folder, f_frame_count=64, l_frame_count=0):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0
    
    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df[(df['matchLogItemType'] == 'RED_JUDGE_BODY_TECH') | (df['matchLogItemType'] == 'BLUE_JUDGE_BODY_TECH') | (df['matchLogItemType'] == 'RED_JUDGE_BODY_TECH') | (df['matchLogItemType'] == 'RED_JUDGE_HEAD_TECH')].iterrows():
        system_time = row['systemTime']

        # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
        video_time_seconds = (system_time - start_system_time) / 1000.0

        # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
        frame_number = int(video_time_seconds * fps)
        frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

        # Belirtilen sayıda önceki frame'leri al
        
        start_frame = max(0, frame_number - f_frame_count)
        end_frame = min(frame_count - 1, frame_number + l_frame_count)
        c = c+1
        # Videoyu oku ve yeni videoya yaz
        output_video = []
        try:
            for i in range(start_frame, end_frame + 1):
                frame = video.get_data(i)
                output_video.append(frame)
        except IndexError as e:
            print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
            c = c-1

        # Klasör kontrolü ve oluşturma
        os.makedirs(output_folder, exist_ok=True)

        # Oluşturulan videoyu belirtilen klasöre kaydet
        output_path = os.path.join(output_folder, f'rotation_judge_{cam_num}_cam_{index}.mkv')
        output_writer = imageio.get_writer(output_path, fps=fps)
        for frame in output_video:
            output_writer.append_data(frame)
        output_writer.close()
    print(f'Yan hakemlerin isaretledikleri rotasyonlu vuruslar {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')

In [33]:

def rotation_scoreboad(video_path, csv_path, output_folder, f_sec=5):
    # CSV dosyasını oku
    df = pd.read_csv(csv_path)

    # Başlangıç sistem zamanını al
    start_system_time = df[df['matchLogItemType'] == 'START_ROUND']['systemTime'].values[0]

    # Videoyu oku
    video = imageio.get_reader(video_path)
    fps = video.get_meta_data()['fps']
    frame_count = video.get_length()
    
    # c degisgeni kaydedilen pozision sayisini tutar
    c = 0
    
    # İlgili frame aralığını bul ve yeni video oluştur
    for index, row in df.iterrows():
        match_log_type = row['matchLogItemType']
        entry_value = row['entryValue']

        if (match_log_type in ['RED_BODY_TECH_POINT', 'RED_HEAD_TECH_POINT', 'BLUE_BODY_TECH_POINT', 'BLUE_HEAD_TECH_POINT']) and entry_value == 'SCOREBOARD_EDITOR':
            system_time = row['systemTime']

            # TIMEOUT satırını bulma
            timeout_row = None
            for i in range(index - 1, -1, -1):
                if df.at[i, 'matchLogItemType'] == 'TIMEOUT':
                    timeout_row = df.iloc[i]
                    break

            if timeout_row is not None:
                timeout_system_time = timeout_row['systemTime']

                # Sistem zamanını milisaniye cinsinden alıp, videonun zaman birimine çeviriyoruz
                video_time_seconds = (timeout_system_time - start_system_time) / 1000.0

                # Videonun zaman birimindeki zamanı frame sayısına çeviriyoruz
                frame_number = int(video_time_seconds * fps)
                frame_number = max(0, min(frame_count - 1, frame_number))  # Frame sayısını kontrol etme

                # Belirtilen sayıda önceki frame'leri al
                start_frame = max(0, frame_number - int(f_sec * fps))
                end_frame = min(frame_count - 1, frame_number)
                c = c+1
                # Videoyu oku ve yeni videoya yaz
                output_video = []
                try:
                    for i in range(start_frame, end_frame + 1):
                        frame = video.get_data(i)
                        output_video.append(frame)
                except IndexError as e:
                    print(f"Hata: {e}. Video uzunluğunu aşan bir indeksle karşılaşıldı. Video işleme devam ediliyor.")
                    c = c-1

                # Klasör kontrolü ve oluşturma
                os.makedirs(output_folder, exist_ok=True)

                # Oluşturulan videoyu belirtilen klasöre kaydet
                output_path = os.path.join(output_folder, f'rotation_scoreboad_{cam_num}_cam_{index}.mkv')
                output_writer = imageio.get_writer(output_path, fps=fps)
                for frame in output_video:
                    output_writer.append_data(frame)
                output_writer.close()
    print(f'Scoreboardan verilen rotasyonlu vuruslar {output_folder} dosyasina kaydedildi. Toplam {c} pozisyon.')

In [34]:
date='05_25'
court_num = 1

#video_konum = f'/home/muhlabws4/videolar konya/cumakort3/Dışa Aktar 12-29-2023 6-48-32 PM/Medya yürütücüsü formatı/CAM-{cam_num}'
video_konum  ='D:/multigames/video'
#video_name = '12_29_2023 5_49_59 PM (UTC+03_00).mkv'
video_name = 'MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 25052024 - 2st DAY - 1 COURT.mp4'
video_path =f'{video_konum}/{video_name}'

#csv_konum = f'/home/muhlabws4/Turkish_Open_2024_Antalya/{date}_2024/{court_num}th_court/matchlog'
csv_konum = 'D:/multigames/csv/Multigames_2024_Ankara/05_25_2024/1th_court/matchlog'
csv_name = '05_25-101-119.csv'
csv_path = f'{csv_konum}/{csv_name}'

output_path = f'D:/multigames/Ham Video/Multigames_2024_Ankara/{date}_2024/{court_num}th_court/rotation'

In [35]:
rotation_judge(video_path, csv_path, f'{output_path}_rotation_judge', f_frame_count=30)

C:\Users\eslemkurt\.conda\envs\en\Lib\site-packages\imageio\plugins\spe.py:507: UserWarning: The file header of D:\multigames\video\MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 25052024 - 2st DAY - 1 COURT.mp4 claims there are 16833816 frames, but there are actually 1 frames.
  warnings.warn(


KeyError: 'fps'

In [12]:
rotation_scoreboad(video_path, csv_path, f'{output_path}_rotation_scoreboard', f_sec=7)

C:\Users\eslemkurt\.conda\envs\en\Lib\site-packages\imageio\plugins\spe.py:507: UserWarning: The file header of D:\multigames\video\MULTI EUROPEAN GAMES TAEKWONDO 2024 - ANKARA - 25052024 - 2st DAY - 1 COURT.mp4 claims there are 16833816 frames, but there are actually 1 frames.
  warnings.warn(


KeyError: 'fps'

In [ ]:
import cv2

video_path = "/path/to/your/video.mp4"

# Videoyu aç
cap = cv2.VideoCapture(video_path)

# FPS değerini al
fps = cap.get(cv2.CAP_PROP_FPS)

# Toplam kare sayısını al
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Videoyu kapat
cap.release()

print(f"FPS Değeri: {fps}")
print(f"Toplam Frame Sayısı: {frame_count}")
